In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from tqdm import tqdm_notebook as tqdm
import requests as req
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 17, 8

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.amazon.com/-/ko/product-reviews/B01N05APQY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
time.sleep(0.5)

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 8.99MB/s]


In [3]:
name_list = []
score_list = []
date_list = []
style_list = []
review_list = []

while True:
    try:
        star = bs(driver.page_source, "html.parser")
        score = star.find_all("i",{'data-hook':"review-star-rating"})
        date = driver.find_elements_by_css_selector('span.a-size-base.a-color-secondary.review-date')
        style = driver.find_elements_by_css_selector('a.a-size-mini.a-link-normal.a-color-secondary')
        review = driver.find_elements_by_css_selector("div.a-row.a-spacing-small.review-data")
        #time.sleep(1)
        for i in score:
            score_list.append(i.text)
        for i in date:
            date_list.append(i.text)
        for i in style:
            style_list.append(i.text)
        for i in review:
            review_list.append(i.text)
        #time.sleep(1)
        btn = driver.find_element_by_css_selector('#cm_cr-pagination_bar > ul > li.a-last')
        btn.click()
        time.sleep(2)
    except:
        break

In [4]:
len(score_list),len(date_list),len(style_list),len(review_list)

(2504, 3024, 2504, 2504)

In [5]:
Coffee1 = {'Star': score_list,'Style': style_list, 'Review': review_list}

In [6]:
Coffee2 = {'Date': date_list}

In [7]:
coffee1 = pd.DataFrame(Coffee1)
coffee2 = pd.DataFrame(Coffee2)

In [8]:
coffee = pd.concat([coffee1, coffee2],axis=1)
coffee.to_csv('homecafe2.csv')

In [52]:
date = pd.read_csv('homecafe2.csv',encoding='utf-8')
cafe = date.head(2482)

In [53]:
cafe.to_csv('cap.csv')

In [54]:
cafe1 = pd.read_csv("cap.csv", encoding = 'utf-8')
cafe1['Star'] = cafe1['Star'].str.extract(r'(\d+.\d+)')
cafe1

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Star,Style,Review,Date
0,0,0,0,5.0,스타일: 베스트 셀러 팩,"I really enjoy this coffee , but, there is not...",Reviewed in the United States 🇺🇸 on 2018년 8월 7일
1,1,1,1,5.0,스타일: 베스트 셀러 팩,You need something quick and to start your day...,Reviewed in the United States 🇺🇸 on 2020년 1월 30일
2,2,2,2,5.0,스타일: 베스트 셀러 팩,It's usually not the freshness I have to worry...,Reviewed in the United States 🇺🇸 on 2022년 9월 23일
3,3,3,3,5.0,스타일: 베스트 셀러 팩,I love a cup of Nespresso. I like coffee but N...,Reviewed in the United States 🇺🇸 on 2022년 9월 16일
4,4,4,4,5.0,스타일: 베스트 셀러 팩,I love that I can quickly make a delicious cup...,Reviewed in the United States 🇺🇸 on 2022년 9월 14일
...,...,...,...,...,...,...,...
2477,2477,2477,2477,5.0,스타일: 베스트 셀러 팩,I like the product,Reviewed in the United States 🇺🇸 on 2021년 6월 28일
2478,2478,2478,2478,5.0,스타일: 베스트 셀러 팩,While I like these capsules and I love my nesp...,Reviewed in the United States 🇺🇸 on 2021년 12월 21일
2479,2479,2479,2479,5.0,스타일: 베스트 셀러 팩,Good product,Reviewed in the United States 🇺🇸 on 2020년 4월 19일
2480,2480,2480,2480,5.0,스타일: 베스트 셀러 팩,arrived in good condition,Reviewed in the United States 🇺🇸 on 2017년 8월 23일


In [55]:
homecafe2 = cafe1['Style'].str.split("색상:", expand = True)
homecafe2[0] = homecafe2[0].str.replace("스타일: ", "")
#homecafe2.rename(columns={0: "Style", 1: "Color"}, inplace=True)
homecafe2.rename(columns={0: "Style"}, inplace=True)
homecafe2

,Style
0,베스트 셀러 팩
1,베스트 셀러 팩
2,베스트 셀러 팩
3,베스트 셀러 팩
4,베스트 셀러 팩
...,...
2477,베스트 셀러 팩
2478,베스트 셀러 팩
2479,베스트 셀러 팩
2480,베스트 셀러 팩


In [56]:
ymd = cafe1['Date'].str.replace("Reviewed in the United States 🇺🇸 on ", "")
ymd = ymd.str.replace("년 ", "-")
ymd = ymd.str.replace("월 ", "-")
ymd = ymd.str.replace("일", "")
ymdf = pd.to_datetime(ymd)
ymdf

0      2018-08-07
1      2020-01-30
2      2022-09-23
3      2022-09-16
4      2022-09-14
          ...    
2477   2021-06-28
2478   2021-12-21
2479   2020-04-19
2480   2017-08-23
2481   2021-08-11
Name: Date, Length: 2482, dtype: datetime64[ns]

In [57]:
homecafe2['Star'] = cafe1['Star']
homecafe2 = homecafe2[['Star','Style']]
homecafe2['Review'] = cafe1['Review']

In [58]:
homecafe2['Date'] = ymdf
homecafe2

,Star,Style,Review,Date
0,5.0,베스트 셀러 팩,"I really enjoy this coffee , but, there is not...",2018-08-07
1,5.0,베스트 셀러 팩,You need something quick and to start your day...,2020-01-30
2,5.0,베스트 셀러 팩,It's usually not the freshness I have to worry...,2022-09-23
3,5.0,베스트 셀러 팩,I love a cup of Nespresso. I like coffee but N...,2022-09-16
4,5.0,베스트 셀러 팩,I love that I can quickly make a delicious cup...,2022-09-14
...,...,...,...,...
2477,5.0,베스트 셀러 팩,I like the product,2021-06-28
2478,5.0,베스트 셀러 팩,While I like these capsules and I love my nesp...,2021-12-21
2479,5.0,베스트 셀러 팩,Good product,2020-04-19
2480,5.0,베스트 셀러 팩,arrived in good condition,2017-08-23


In [59]:
cap = homecafe2[['Star','Date','Style','Review']]
cap

,Star,Date,Style,Review
0,5.0,2018-08-07,베스트 셀러 팩,"I really enjoy this coffee , but, there is not..."
1,5.0,2020-01-30,베스트 셀러 팩,You need something quick and to start your day...
2,5.0,2022-09-23,베스트 셀러 팩,It's usually not the freshness I have to worry...
3,5.0,2022-09-16,베스트 셀러 팩,I love a cup of Nespresso. I like coffee but N...
4,5.0,2022-09-14,베스트 셀러 팩,I love that I can quickly make a delicious cup...
...,...,...,...,...
2477,5.0,2021-06-28,베스트 셀러 팩,I like the product
2478,5.0,2021-12-21,베스트 셀러 팩,While I like these capsules and I love my nesp...
2479,5.0,2020-04-19,베스트 셀러 팩,Good product
2480,5.0,2017-08-23,베스트 셀러 팩,arrived in good condition


In [60]:
cap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Star    2482 non-null   object        
 1   Date    2482 non-null   datetime64[ns]
 2   Style   2482 non-null   object        
 3   Review  2474 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 77.7+ KB


In [61]:
cap.to_csv('capsule.csv',encoding='utf-8-sig')

In [64]:
date = pd.read_csv('capsule.csv',encoding='utf-8-sig')
date

,Unnamed: 0,Star,Date,Style,Review
0,0,5.0,2018-08-07,베스트 셀러 팩,"I really enjoy this coffee , but, there is not..."
1,1,5.0,2020-01-30,베스트 셀러 팩,You need something quick and to start your day...
2,2,5.0,2022-09-23,베스트 셀러 팩,It's usually not the freshness I have to worry...
3,3,5.0,2022-09-16,베스트 셀러 팩,I love a cup of Nespresso. I like coffee but N...
4,4,5.0,2022-09-14,베스트 셀러 팩,I love that I can quickly make a delicious cup...
...,...,...,...,...,...
2477,2477,5.0,2021-06-28,베스트 셀러 팩,I like the product
2478,2478,5.0,2021-12-21,베스트 셀러 팩,While I like these capsules and I love my nesp...
2479,2479,5.0,2020-04-19,베스트 셀러 팩,Good product
2480,2480,5.0,2017-08-23,베스트 셀러 팩,arrived in good condition


In [65]:
date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2482 non-null   int64  
 1   Star        2482 non-null   float64
 2   Date        2482 non-null   object 
 3   Style       2482 non-null   object 
 4   Review      2474 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 97.1+ KB
